유튜브: https://youtu.be/Db1UpxNnXoY

# KIND 시장조치 스크래핑

In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd

In [44]:
stock_code = '028300'
list_1 = []
list_2 = []

driver = webdriver.Chrome()
url = 'https://kind.krx.co.kr/main.do?method=loadInitPage&scrnmode=1'
driver.get(url)

driver.find_element_by_css_selector('#AKCKwdTop').send_keys(stock_code)
driver.find_element_by_css_selector('#AKCKwdTop').send_keys(Keys.ENTER)
time.sleep(3)

driver.find_element_by_css_selector('body > section.wrapper-contents > section > article > nav > ul > li:nth-child(3) > a').click()
time.sleep(3)

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

dates = soup.find_all('td', class_='first')
titles = soup.find_all(id='disc')

for date in dates:
    list_1.append(date.text)
    
for title in titles:
    list_2.append(title['title'])
    
driver.quit()

df_kind = pd.DataFrame(list(zip(list_1, list_2)), columns=['일자', '제목'])
df_kind.head()

,일자,제목
0,2022-05-13 18:09,공매도 과열종목 지정(공매도 거래 금지 적용)
1,2022-03-17 16:03,[투자주의]특정계좌(군) 매매관여 과다종목
2,2022-01-13 18:29,불성실공시법인지정(공시불이행 2건)
3,2022-01-11 18:30,기타시장안내(약명 및 영문명)
4,2021-12-22 18:03,불성실공시법인지정예고


In [64]:
df_kind = df_kind[df_kind['제목'].str.contains('공매도')]
df_kind.head()

,일자,제목
0,2022-05-13 18:09,공매도 과열종목 지정(공매도 거래 금지 적용)
10,2021-08-31 18:13,공매도 과열종목 지정(공매도 거래 금지 적용)
13,2021-08-18 18:14,공매도 과열종목 지정(공매도 거래 금지 적용)
14,2021-07-15 18:16,공매도 과열종목 지정(공매도 거래 금지 적용)


### str → date

In [65]:
df_kind['일자'] = pd.to_datetime(df_kind['일자']).apply(lambda x : x.date())
df_kind = df_kind[1:]
df_kind.head()

,일자,제목
10,2021-08-31,공매도 과열종목 지정(공매도 거래 금지 적용)
13,2021-08-18,공매도 과열종목 지정(공매도 거래 금지 적용)
14,2021-07-15,공매도 과열종목 지정(공매도 거래 금지 적용)


# 다음날 주가는?

In [66]:
import FinanceDataReader as fdr
from datetime import timedelta

In [67]:
df_finance = fdr.DataReader(symbol=stock_code)
df_finance.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-05-09,30050,30700,29450,29500,572919,-0.015025
2022-05-10,29000,29600,28850,28900,605682,-0.020339
2022-05-11,28900,31100,28500,31000,1007582,0.072664
2022-05-12,30600,36900,30550,33800,5484643,0.090323
2022-05-13,39000,43900,34950,42500,10453103,0.257396


In [68]:
Change = []
for i in range(len(df_kind)):
    tommorrow_index = df_kind['일자'].iloc[i] + timedelta(days=1)
    change = df_finance.loc[tommorrow_index]['Change']
    change = round(change*100,2)
    Change.append(change)

df_kind['다음날 종가등락률'] = Change
df_kind

,일자,제목,다음날 종가등락률
10,2021-08-31,공매도 과열종목 지정(공매도 거래 금지 적용),9.01
13,2021-08-18,공매도 과열종목 지정(공매도 거래 금지 적용),-1.67
14,2021-07-15,공매도 과열종목 지정(공매도 거래 금지 적용),-3.77
